In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [74]:
df = pd.read_csv(
    'data/data_4_aggregated_day.csv',
    index_col=0)

In [75]:
# this is the size of the sliding window: the data of N consecutive days will be used to predict the mood of the next day
N = 3

data_new = df.copy()

data_new['datetime'] = pd.to_datetime(data_new['datetime'])

# Ensure data is sorted as expected
data_new.sort_values(by=['id', 'datetime'], inplace=True)

In [76]:
# Function to filter for consecutive days
def filter_consecutive_days(group):
    # Calculate the difference between consecutive days
    group['diff_days'] = group['datetime'].diff().dt.days

    # Identify start of new consecutive sequence
    group['start_new_sequence'] = (group['diff_days'] != 1).cumsum()

    # Count days in each sequence
    group['sequence_count'] = group.groupby('start_new_sequence').cumcount() + 1

    # Filter sequences with at least n consecutive days
    valid_sequences = group[group['sequence_count'] >= N]['start_new_sequence'].unique()
    return group[group['start_new_sequence'].isin(valid_sequences)]

# Apply the function to each group
consecutive_data = data_new.groupby('id').apply(filter_consecutive_days).reset_index(drop=True)

# Drop auxiliary columns used for calculation
consecutive_data.drop(columns=['diff_days', 'start_new_sequence', 'sequence_count'], inplace=True)

# Display the processed data
consecutive_data.head(), consecutive_data['datetime'].min(), consecutive_data['datetime'].max()

# consecutive_data.to_csv('data_new.csv', index=False)

(   id   datetime  observation_day  mood  arousal  valence  activity  \
 0   1 2014-03-22                1   6.4      0.6      0.6  0.265094   
 1   1 2014-03-23                2   6.4      0.1      0.3  0.171456   
 2   1 2014-03-24                3   6.0      0.8      0.0  0.124599   
 3   1 2014-03-25                4   6.5      0.0      0.5  0.148496   
 4   1 2014-03-26                5   6.6     -0.2      0.6  0.154535   
 
          screen  call  sms  ...  appCat.utilities  appCat.weather  \
 0   6005.737999     3    1  ...           117.621           0.000   
 1   4122.447001     0    0  ...             0.000          30.386   
 2  12245.631000    10    0  ...           178.732           0.000   
 3  12254.188999     0    1  ...            74.431           0.000   
 4   9867.717000     0    0  ...             0.000           0.000   
 
    night_physical_activity  night_screen  night_appcat_combined  \
 0                 0.000000    370.382000                378.791   
 1      

In [77]:
data_new.columns

Index(['id', 'datetime', 'observation_day', 'mood', 'arousal', 'valence',
       'activity', 'screen', 'call', 'sms', 'appCat.builtin',
       'appCat.communication', 'appCat.entertainment', 'appCat.finance',
       'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
       'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather',
       'night_physical_activity', 'night_screen', 'night_appcat_combined',
       'SUM_inactivity_timediff', 'weekday', 'week_of_year', 'appCat_total',
       'mood_of_next_day'],
      dtype='object')

In [78]:
mean_cols = ['mood', 'arousal',
       'valence', 'activity', "night_physical_activity", "SUM_inactivity_timediff"]
sum_cols = ['screen', 'call', 'sms',
       'appCat_total', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 
       "night_screen", "night_appcat_combined"]

In [79]:
# Function to apply rolling window aggregation with correct mood_of_next_day handling
def sliding_window_aggregation_corrected(group):
    # Rolling window setup for means and sums
    rolled_means = group[mean_cols].rolling(window=N, min_periods=N).mean()
    rolled_sums = group[sum_cols].rolling(window=N, min_periods=N).sum()

    # Combine the results into the day N row
    combined = pd.concat([rolled_means, rolled_sums], axis=1)

    # The mood_of_next_day from day n's record
    combined['mood_of_next_day'] = group['mood_of_next_day']

    # We align this so that the aggregates and mood_of_next_day are all in the day N row
    return combined

# Apply the function to each group
sliding_window_data_corrected = (consecutive_data.groupby('id').apply(sliding_window_aggregation_corrected)
                                 .reset_index(level=0, drop=False))

# Drop rows where the window did not complete, which is the first N-1 days of each sequence
sliding_window_data_final = sliding_window_data_corrected.dropna().reset_index(drop=False)

sliding_window_data_final.drop(['index'], axis=1, inplace=True)
# Display the first few rows of the processed data
sliding_window_data_final.head()

# sliding_window_data_final.to_csv('data_sliding_window.csv', index=False)

,id,mood,arousal,valence,activity,night_physical_activity,SUM_inactivity_timediff,screen,call,sms,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,night_screen,night_appcat_combined,mood_of_next_day
0,1,6.266667,0.500000,0.300000,0.187050,0.005343,6.655667,22373.816000,13.0,1.0,...,3.010,182.212,3965.580,457.110,0.000,296.353,30.386,3177.074001,3185.304,6.5
1,1,6.300000,0.300000,0.266667,0.148183,0.006310,5.989000,28622.267000,10.0,1.0,...,3.010,225.740,4988.079,419.805,235.223,253.163,30.386,5608.306001,5602.079,6.6
2,1,6.366667,0.200000,0.366667,0.142543,0.009404,4.666667,34367.537000,10.0,1.0,...,3.010,269.585,7556.619,419.805,235.223,253.163,0.000,14841.371001,13670.931,7.0
3,1,6.700000,-0.200000,0.566667,0.201463,0.010425,4.333333,39328.485999,2.0,2.0,...,182.451,265.579,5333.648,47.314,235.223,238.372,0.000,18582.813000,15697.366,6.5
4,1,6.700000,-0.433333,0.600000,0.201087,0.009459,3.666667,31399.294000,7.0,1.0,...,182.451,351.869,4384.258,1180.323,0.000,465.658,0.000,16430.686000,23557.885,7.1


In [80]:
# Classification KNN
sliding_window_data_final_classification = sliding_window_data_final.copy()
# Preparing the target variable
# Categorizing mood_of_next_day
def categorize_mood(mood):
    if 1.0 <= mood <= 6.9999999:
        return 1  # Mood between 1 and 7
    elif 7.0 <= mood <= 7.9999999:
        return 2  # Mood between 7 and 8
    elif 8.0 <= mood <= 10.0:
        return 3  # Mood between 8 and 10
    else:
        return None  # In case there are values outside the expected range

# Apply the categorization function to the 'mood_of_next_day' column
# CAREFUL: RUN THIS ONLY ONCE, OTHERWISE IT WILL KEEP CATEGORIZING THE ALREADY CATEGORIZED VALUES
sliding_window_data_final_classification['mood_of_next_day'] = sliding_window_data_final_classification['mood_of_next_day'].apply(categorize_mood)

In [81]:
sliding_window_data_final_classification.head()

,id,mood,arousal,valence,activity,night_physical_activity,SUM_inactivity_timediff,screen,call,sms,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,night_screen,night_appcat_combined,mood_of_next_day
0,1,6.266667,0.500000,0.300000,0.187050,0.005343,6.655667,22373.816000,13.0,1.0,...,3.010,182.212,3965.580,457.110,0.000,296.353,30.386,3177.074001,3185.304,1
1,1,6.300000,0.300000,0.266667,0.148183,0.006310,5.989000,28622.267000,10.0,1.0,...,3.010,225.740,4988.079,419.805,235.223,253.163,30.386,5608.306001,5602.079,1
2,1,6.366667,0.200000,0.366667,0.142543,0.009404,4.666667,34367.537000,10.0,1.0,...,3.010,269.585,7556.619,419.805,235.223,253.163,0.000,14841.371001,13670.931,2
3,1,6.700000,-0.200000,0.566667,0.201463,0.010425,4.333333,39328.485999,2.0,2.0,...,182.451,265.579,5333.648,47.314,235.223,238.372,0.000,18582.813000,15697.366,1
4,1,6.700000,-0.433333,0.600000,0.201087,0.009459,3.666667,31399.294000,7.0,1.0,...,182.451,351.869,4384.258,1180.323,0.000,465.658,0.000,16430.686000,23557.885,2


In [72]:
# sliding_window_data_final_classification.drop('id', axis=1, inplace=True)

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming you have a 'participant_id' column in your dataframe
participant_ids = sliding_window_data_final_classification['id'].unique()

# Now split the participant IDs into training and test sets
train_ids, test_ids = train_test_split(participant_ids, test_size=0.2, random_state=42)

# Split the data into training and test sets based on participant IDs
X_train = sliding_window_data_final_classification[sliding_window_data_final_classification['id'].isin(train_ids)]
y_train = X_train.pop('mood_of_next_day')

X_test = sliding_window_data_final_classification[sliding_window_data_final_classification['id'].isin(test_ids)]
y_test = X_test.pop('mood_of_next_day')

# If you want to remove the 'participant_id' column from the features, do it here
X_train = X_train.drop('id', axis=1)
X_test = X_test.drop('id', axis=1)

# Scale the features using StandardScaler
# Fit the scaler on the training data and transform both the training and test data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Note: You should only fit the scaler on the training data to prevent data leakage


In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score


# Split the data into features (X) and target (y)
X = sliding_window_data_final_classification.drop('mood_of_next_day', axis=1)
y = sliding_window_data_final_classification['mood_of_next_day']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [84]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [85]:
y_pred = knn.predict(X_test)

In [86]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4126984126984127


In [87]:
# Classification RFC
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [88]:
y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5238095238095238


In [54]:
from sklearn.metrics import precision_score, recall_score

y_pred = rfc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.6936416184971098
Precision: 0.6804124094440203
Recall: 0.6936416184971098


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
# Regression
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

sliding_window_data_final_regression = sliding_window_data_final.copy()
sliding_window_data_final_regression.head()

,id,mood,arousal,valence,activity,night_physical_activity,SUM_inactivity_timediff,screen,call,sms,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,night_screen,night_appcat_combined,mood_of_next_day
0,1,6.400000,0.533333,0.500000,0.187050,0.005343,6.655667,22373.816000,13.0,1.0,...,3.010,182.212,3965.580,457.110,0.000,296.353,30.386,3177.074001,3185.304,7.0
1,1,6.600000,0.266667,0.400000,0.148183,0.006310,5.989000,28622.267000,10.0,1.0,...,3.010,225.740,4988.079,419.805,235.223,253.163,30.386,5608.306001,5602.079,6.6
2,1,6.533333,0.133333,0.366667,0.142543,0.009404,4.666667,34367.537000,10.0,1.0,...,3.010,269.585,7556.619,419.805,235.223,253.163,0.000,14841.371001,13670.931,7.0
3,1,6.866667,-0.133333,0.633333,0.201463,0.010425,4.333333,39328.485999,2.0,2.0,...,182.451,265.579,5333.648,47.314,235.223,238.372,0.000,18582.813000,15697.366,6.7
4,1,6.766667,-0.266667,0.733333,0.201087,0.009459,3.666667,31399.294000,7.0,1.0,...,182.451,351.869,4384.258,1180.323,0.000,465.658,0.000,16430.686000,23557.885,7.5


In [58]:
sliding_window_data_final_regression.drop('id', axis=1, inplace=True)

In [59]:
# Split the data into features (X) and target (y)
X = sliding_window_data_final_regression.drop('mood_of_next_day', axis=1)
y = sliding_window_data_final_regression['mood_of_next_day']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [60]:
# Training regressors
reg1 = GradientBoostingRegressor(random_state=42)
reg2 = RandomForestRegressor(random_state=42)
reg3 = LinearRegression()

reg1.fit(X, y)
reg2.fit(X, y)
reg3.fit(X, y)


ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(X, y)

In [61]:
xt = X[:20]

pred1 = reg1.predict(xt)
pred2 = reg2.predict(xt)
pred3 = reg3.predict(xt)
pred4 = ereg.predict(xt)

In [62]:
from sklearn.metrics import mean_absolute_error, r2_score

# Model evaluation
# Make predictions on the testing data
pred = ereg.predict(X_test)
 
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, pred)
 
# Calculate R-squared (R2) score
r2 = r2_score(y_test, pred)
 
# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2) Score: {r2:.2f}')

Mean Absolute Error (MAE): 0.27
R-squared (R2) Score: 0.66
